In [3]:
!pip install pandas requests openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import pandas as pd
import requests
import os

# Set paths
csv_path = 'C:/Users/ICB_Server/Cloudy_WebDev/database/dataset.csv'
output_csv_path = 'C:/Users/ICB_Server/Cloudy_WebDev/database/cleaned_dataset.csv'
image_folder = 'C:/Users/ICB_Server/Cloudy_WebDev/database/pictures'

if not os.path.exists(image_folder):
    os.makedirs(image_folder)

def download_image(image_url, sku, index):
    filename = f"{sku}_{index + 1}.jpg"
    filepath = os.path.join(image_folder, filename)

    try:
        response = requests.get(image_url, stream=True, timeout=10)
        if response.status_code == 200:
            with open(filepath, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            return filename
        else:
            print(f"Failed to download (status {response.status_code}): {image_url}")
            return None
    except Exception as e:
        print(f"Error downloading {image_url}: {e}")
        return None
df = pd.read_csv(csv_path)
rows_to_keep = []

for _, row in df.iterrows():
    image_urls = str(row['images']).split('~')
    downloaded_images = []
    all_success = True

    for index, url in enumerate(image_urls):
        filename = download_image(url, row['sku'], index)
        if filename:
            downloaded_images.append(filename)
        else:
            all_success = False
            break

    if all_success:
        row['images'] = '~'.join(downloaded_images)
        rows_to_keep.append(row)

df_valid = pd.DataFrame(rows_to_keep)
df_valid.to_csv(output_csv_path, index=False)


Failed to download (status 404): https://assets.adidas.com/images/w_600,f_auto,q_auto/adce4c860bd841e0a853aafd00b362d7_9366/Beach_Shorts_Black_FJ5089_21_model.jpg
Failed to download (status 404): https://assets.adidas.com/images/w_600,f_auto,q_auto/0ca38931b0cb4d14a3daac440068e9bd_9366/Mesh_Broken-Stripe_Polo_Shirt_Blue_GM0239_21_model.jpg
Failed to download (status 404): https://assets.adidas.com/images/w_600,f_auto,q_auto/680d1a3589b54d1da173ac32014b9a37_9366/Adicross_Hybrid_Shorts_Black_GM5505_21_model.jpg
Failed to download (status 404): https://assets.adidas.com/images/w_600,f_auto,q_auto/f8b3261ac689442f8ecfabd601157e2e_9366/Classic_3-Stripes_Swimsuit_Black_FS3923_21_model.jpg
Failed to download (status 404): https://assets.adidas.com/images/w_600,f_auto,q_auto/5aee09254c2446da9ed0ac560162c59e_9366/Tiro_21_Windbreaker_Black_GP4967_21_model.jpg
Failed to download (status 404): https://assets.adidas.com/images/w_600,f_auto,q_auto/99a118e2f9cb482db9d9ac5d0022dde3_9366/Formotion_Scul

https://www.kaggle.com/datasets/heemalichaudhari/adidas-sales-dataset

In [ ]:
import pandas as pd
import sqlite3
import os

# Paths
db_path = 'C:/Users/ICB_Server/Cloudy_WebDev/database/database.sqlite'
csv_path = 'C:/Users/ICB_Server/Cloudy_WebDev/database/dataset.csv'

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

schema_sql = '''
DROP TABLE IF EXISTS DetailPage;
DROP TABLE IF EXISTS ImageHomepage;
DROP TABLE IF EXISTS Homepage;
DROP TABLE IF EXISTS Profile;
DROP TABLE IF EXISTS ProductInfo;
DROP TABLE IF EXISTS Product;
DROP TABLE IF EXISTS Admin;

CREATE TABLE Admin (
    Email TEXT PRIMARY KEY,
    Password TEXT NOT NULL
);

CREATE TABLE Product (
    ProductID TEXT PRIMARY KEY,
    ProductName TEXT NOT NULL,
    FullProductName TEXT NOT NULL,
    Details TEXT,
    Description TEXT,
    Price INTEGER NOT NULL,
    Category TEXT
);

CREATE TABLE ProductInfo (
    Image TEXT,
    ProductID TEXT,
    FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
);

CREATE TABLE Profile (
    FirstName TEXT,
    LastName TEXT,
    Dob TEXT,
    Address TEXT,
    Email TEXT,
    Photo TEXT
);

CREATE TABLE Homepage (
    Title TEXT NOT NULL,
    Brand TEXT,
    ProductID TEXT,
    FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
);

CREATE TABLE ImageHomepage (
    Types TEXT NOT NULL,
    Image TEXT NOT NULL
);

CREATE TABLE DetailPage (
    keyFeatures TEXT,
    MaterialDetail TEXT,
    SizeAndFit TEXT
);
'''

cursor.executescript(schema_sql)
df = pd.read_csv(csv_path)
print(f"Loading :)))")
inserted = 0
for _, row in df.iterrows():
    try:
        product_id = str(row['sku']).strip()
        product_name = str(row['name']).strip()
        full_name = str(row['breadcrumbs']).strip()
        details = str(row['category']).strip()
        description = str(row['description']).strip()
        price = int(float(row['selling_price'])) if not pd.isna(row['selling_price']) else 0
        category = str(row['category']).strip()

        cursor.execute('''
            INSERT OR REPLACE INTO Product 
            (ProductID, ProductName, FullProductName, Details, Description, Price, Category)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (product_id, product_name, full_name, details, description, price, category))

        images = str(row['images']).split('~')
        for image_url in images:
            image_url = image_url.strip()
            if image_url:
                cursor.execute('''
                    INSERT INTO ProductInfo (Image, ProductID) VALUES (?, ?)
                ''', (image_url, product_id))

        inserted += 1
    except Exception as e:
        print(f"Error")

conn.commit()
conn.close()
print(f"Done")

Loading :)))
Done
